In [57]:
import re
import csv

def extract_bytes_from_c_file(filename):
    with open(filename, 'r') as file:
        c_code = file.read()

    packet_pattern = r'static const unsigned char (\w+)\[(\d+)\] = \{(.*?)\};'
    packets = re.findall(packet_pattern, c_code, re.DOTALL)

    packet_byte_lists = []
    for packet_name, packet_size, packet_data in packets:
        
        bytes_str = re.sub(r'\/\*.*?\*\/', '', packet_data)  # Remove comments
        bytes_list = re.findall(r'0x\w\w', bytes_str)  # Find byte strings
        byte_list = [int(byte_str, 16) for byte_str in bytes_list]
        
        packet_byte_lists.append(bytes(byte_list))

    return packet_byte_lists

In [58]:
# First extract all packets as bytes
filename = 'packets.c'
packet_byte_lists = extract_bytes_from_c_file(filename)
print(f"Extracted {len(packet_byte_lists)} packets from {filename}")

Extracted 31195 packets from packets.c


In [37]:
# Now also parse the json
import json
filename = 'packets.json'
with open(filename, 'r') as file:
    packets_metadata = json.load(file)

print(f"Extracted {len(packets_metadata)} packets from {filename}")

assert len(packet_byte_lists) == len(packets_metadata)

Extracted 31195 packets from packets.json


In [39]:
def parse_datagram(data):
  
  if "ecat" not in data["_source"]["layers"].keys():
    return []
  
  result = []
  
  for gram in data["_source"]["layers"]["ecat"]:
    
    gram["Header"]
  
  
  print(ecat_datagram)

parse_datagram(packets_metadata[1])  

TypeError: string indices must be integers

In [96]:
def byte_string_to_c_notation(byte_string):
    c_notation = "0x"
    for byte in byte_string:
        c_notation += format(byte, '02x')
    return c_notation

In [132]:
from collections import deque

master = b'\x70\x45\xa9'

commands = {
  "8": "BWR",
  "7": "BRD",
  "1": "APRD",
}

code = """#include <stdio.h>
#include <assert.h>
#include <string.h>
#include <inttypes.h>

#include "ethercat.h"

void setup() {
    int retval = 0;
"""

sent_frames_not_resolved = deque([])

for i, raw_packet in enumerate(packet_byte_lists[:11]):
  
  # Only want ethercat frames
  if raw_packet[12:14] != b'\x88\xa4':
    continue
  
  is_master = raw_packet[9:12] == master  
  
  # Now parse the packet, which might exist of multiple frames
  
  print("Frame ", i)
  
  still_frames_left = True
  offset = 16
  frames = []
  while still_frames_left:

    cmd = commands[str(raw_packet[offset])]
    adp = byte_string_to_c_notation(raw_packet[offset+2:offset+4][::-1])
    ado = byte_string_to_c_notation(raw_packet[offset+4:offset+6][::-1])
    length = int(raw_packet[offset + 6]) + int(raw_packet[offset + 7] & 0x70) * 2**8

    data = [byte_string_to_c_notation([b]) for b in raw_packet[offset + 10:offset + 10 + length][::-1]]
    readable_data = byte_string_to_c_notation(raw_packet[offset + 10:offset + 10 + length][::-1])

    working_counter = int(raw_packet[offset + 10 + length]) + int(raw_packet[offset + 11 + length]) * 2**8
    
    print(f"{'M' if is_master else 'S'} cmd: {cmd}, adp: {adp}, ado: {ado}, length: {length}, working counter: {working_counter}, data: {data}")
    
    frames.append({
      "cmd": cmd,
      "adp": adp,
      "ado": ado,
      "readable_data": readable_data,
      "length": length,
      "working_counter": working_counter,
      "data": data
    })
    
    still_frames_left = raw_packet[offset + 7] & 0x80  
    offset += 4 + length + 8 
  
  # If the packet comes from the master, then we store it for later as we don't know the answer yet
  if is_master:
    
    # Store frames for later (so that we can 'untangle' the frames later on)
    for frame in frames:
      sent_frames_not_resolved.append(frame)
      
  else:
    
    # If the packet is from the slave, we can match them to the original master packet
    
    for j, slave_frame in enumerate(frames):
    
      master_frame = sent_frames_not_resolved.popleft()
      
      working_counter_difference = slave_frame["working_counter"] - master_frame["working_counter"]
 
      if master_frame["cmd"] == "BWR":
        code += f"""        
    uint8_t data{i}_{j}[{master_frame["length"]}] = {{{", ".join(master_frame["data"])}}};
    retval = ec_BWR({master_frame["adp"]}, {master_frame["ado"]}, {master_frame["length"]}, data{i}_{j}, EC_TIMEOUTSAFE);
    printf(\"{master_frame["cmd"]} {master_frame["ado"]} TX: {master_frame["readable_data"]} \\n \");
    assert(retval == {working_counter_difference});
        """
      elif master_frame["cmd"] == "BRD" or master_frame["cmd"] == "APRD":
        code += f"""        
    uint8_t data{i}_{j}[{master_frame["length"]}];
    uint8_t target_data{i}_{j}[{slave_frame["length"]}] = {{{", ".join(slave_frame["data"])}}};
    retval = ec_{master_frame["cmd"]}({master_frame["adp"]}, {master_frame["ado"]}, {master_frame["length"]}, data{i}_{j}, EC_TIMEOUTSAFE);
    printf(\"{master_frame["cmd"]} {master_frame["ado"]} TX: {master_frame["readable_data"]} RX?: {slave_frame["readable_data"]} \\n \");
    assert(retval == {working_counter_difference});;
    for (int i = 0; i < {master_frame["length"]}; i++) {{
      assert(data{i}_{j}[i] == target_data{i}_{j}[i]);
    }}
        """
      else:
        raise Exception("Not implemented")
      
code += """
}

int main(int argc, char *argv[])
{

   if (argc <= 2) {
      ec_adaptert * adapter = NULL;
      printf("Usage: simple_test ifname1\\nifname = eth0 for example\\n");

      printf ("\\nAvailable adapters:\\n");
      adapter = ec_find_adapters ();
      while (adapter != NULL)
      {
         printf ("    - %s  (%s)\\n", adapter->name, adapter->desc);
         adapter = adapter->next;
      }
      ec_free_adapters(adapter);

      return -1;
   }

   char *ifname = argv[1];

   // Init socket
   if (!ec_init(ifname))
   {
      printf("No socket connection on %s\\nExecute as root\\n", ifname);
      return -1;
   }

   setup();

   ec_close();
   return 0;
}
"""

# Export code to .c file
filename = 'out.c'
f = open(filename, "w")
f.write(code)
f.close()

Frame  1
M cmd: BWR, adp: 0x0000, ado: 0x0101, length: 1, working counter: 0, data: ['0x00']
Frame  2
S cmd: BWR, adp: 0x0001, ado: 0x0101, length: 1, working counter: 1, data: ['0x00']
Frame  3
M cmd: BWR, adp: 0x0000, ado: 0x0101, length: 1, working counter: 0, data: ['0x00']
Frame  4
S cmd: BWR, adp: 0x0001, ado: 0x0101, length: 1, working counter: 1, data: ['0x00']
Frame  5
M cmd: BRD, adp: 0x0000, ado: 0x0130, length: 2, working counter: 0, data: ['0x00', '0x00']
Frame  6
S cmd: BRD, adp: 0x0001, ado: 0x0130, length: 2, working counter: 1, data: ['0x00', '0x01']
Frame  7
M cmd: BWR, adp: 0x0000, ado: 0x0300, length: 8, working counter: 0, data: ['0x00', '0x00', '0x00', '0x00', '0x00', '0x00', '0x00', '0x00']
M cmd: BRD, adp: 0x0000, ado: 0x0130, length: 2, working counter: 0, data: ['0x00', '0x00']
Frame  8
M cmd: APRD, adp: 0x0000, ado: 0x0000, length: 10, working counter: 0, data: ['0x00', '0x00', '0x00', '0x00', '0x00', '0x00', '0x00', '0x00', '0x00', '0x00']
M cmd: APRD, adp: 